https://tproger.ru/translations/markov-chains/

# Цепи Маркова для автоматической генерации текcтов

## Структура данных Dictogram

Dictogram (dict — встроенный тип данных словарь в Python) будет отображать зависимость между звеньями и их частотой появления в тексте, то есть их распределение. Но при этом она будет обладать нужным нам свойством словаря — время выполнения программы не будет зависеть от объема входных данных, а это значит, мы создаем эффективный алгоритм.

In [2]:
import random

class Dictogram(dict):
    def __init__(self, iterable=None):
        # Инициализируем наше распределение как новый объект класса, 
        # добавляем имеющиеся элементы
        super(Dictogram, self).__init__()
        self.types = 0  # число уникальных ключей в распределении
        self.tokens = 0  # общее количество всех слов в распределении
        if iterable:
            self.update(iterable)

    def update(self, iterable):
        # Обновляем распределение элементами из имеющегося 
        # итерируемого набора данных
        for item in iterable:
            if item in self:
                self[item] += 1
                self.tokens += 1
            else:
                self[item] = 1
                self.types += 1
                self.tokens += 1

    def count(self, item):
        # Возвращаем значение счетчика элемента, или 0
        if item in self:
            return self[item]
        return 0

    def return_random_word(self):
        random_key = random.sample(self, 1)
        # Другой способ:
        # random.choice(histogram.keys())
        return random_key[0]

    def return_weighted_random_word(self):
        # Сгенерировать псевдослучайное число между 0 и (n-1),
        # где n - общее число слов
        random_int = random.randint(0, self.tokens-1)
        index = 0
#         list_of_keys = 
        # вывести 'случайный индекс:', random_int
        for key in self.keys():
            index += self[key]
            # вывести индекс
            if(index > random_int):
                # вывести list_of_keys[i]
                return key

В конструктор структуре Dictogram можно передать любой объект, по которому можно итерироваться. Элементами этого объекта будут слова для инициализации Dictogram, например, все слова из какой-нибудь книги. В данном случае мы ведем подсчет элементов, чтобы для обращения к какому-либо из них не нужно было пробегать каждый раз по всему набору данных.

Мы также сделали две функции для возврата случайного слова. Одна функция выбирает случайный ключ в словаре, а другая, принимая во внимание число появлений каждого слова в тексте, возвращает нужное нам слово.

## Структура цепи Маркова

In [3]:
def make_markov_model(data):
    markov_model = dict()

    for i in range(0, len(data)-1):
        if data[i] in markov_model:
            # Просто присоединяем к уже существующему распределению
            markov_model[data[i]].update([data[i+1]])
        else:
            markov_model[data[i]] = Dictogram([data[i+1]])
    return markov_model

В реализации выше у нас есть словарь, который хранит окна в качестве ключа в паре «(ключ, значение)» и распределения в качестве значений в этой паре.

## Парсинг модели

Отлично, мы реализовали словарь. Но как теперь совершить генерацию контента, основываясь на текущем состоянии и шаге к следующему состоянию? Пройдемся по нашей модели:

In [4]:
from collections import deque
import re


def generate_random_start(model):
    # Чтобы сгенерировать любое начальное слово, раскомментируйте строку:
    # return random.choice(model.keys())

    # Чтобы сгенерировать "правильное" начальное слово, используйте код ниже:
    # Правильные начальные слова - это те, что являлись началом предложений в корпусе
    if 'END' in model:
        seed_word = 'END'
        while seed_word == 'END':
            seed_word = model['END'].return_weighted_random_word()
        return seed_word
    return random.choice(list(model.keys()))


def generate_random_sentence(length, markov_model):
    current_word = generate_random_start(markov_model)
    sentence = [current_word]
    for i in range(0, length):
        current_dictogram = markov_model[current_word]
        random_weighted_word = current_dictogram.return_weighted_random_word()
        current_word = random_weighted_word
        sentence.append(current_word)
    sentence[0] = sentence[0].capitalize()
    return ' '.join(sentence) + '.'
    return sentence

Лучше генерировать «правильные» начальные слова – те, которые и в исходном тексте стояли в начале предложения. Для этого мы в словаре находим все ключи «END» и выбираем слово, следующее за одним из них. После генерации начального слова мы ищем, какое слово может идти дальше, обращаясь к тому же словарю, и выбираем нужное на основании комбинации вероятности и случайности. Продолжаем это делать, пока предложение не достигнет установленной нами длины, и в конце возвращаем его. 

## Примеры

In [5]:
text = open('nietzsche.txt').read()
text = text.replace("»", " END").replace("«", "").replace("\n", " ").replace(".", " END").replace(",", "")
frags = [f.lower() if f != 'END' else f for f in text.split(" ") if f]
print(len(frags))

101895


### Для 1-го порядка

In [6]:
model = make_markov_model(frags)

In [14]:
s = 0
for i in model['that']: s+= model['that'][i]
s


1307

In [7]:
generate_random_sentence(100, model).replace(" END", ".")

'And support from every one"; a "will to my taste end" he has been mistaken about germans.. "i like the earliest alliances: the end of the contemplation of its intellectual superiority in the language shows herself by his shame has on the cross themselves by their conduct and injury and man is the general tendency end" 4. 55 =ethic discredited. byron sound in the same as with perfect itself beyond the final completeness. 218. oh! friends and love of depth lambent airs it is very useful. but who have hitherto existed--a notion which conduct.'

## Задание. Cтруктура цепи Маркова N-го порядка
Допишите функцию ниже для учета предысторий длины 3 (то есть, порядка (order) = 4). 
Функция состоит из цикла, который:
* создает 3-грамы
* добавляет их в словарь

In [ ]:
def make_higher_order_markov_model(order, data):
    markov_model = dict()
    # ваш код здесь
    return markov_model

In [ ]:
model = make_higher_order_markov_model(4, frags)

In [ ]:
def triplet(model, w=''): # функция генерирует случайный первый триплет, начинающийся на END
    s = []
    if w == 'END':
        for i in model:
            if i[0] == 'END':
                s.append(i)
        n = random.randrange(len(s))
        return(s[n])
    else:
        n = random.randrange(len(list(model.keys())))
        return(list(model.keys())[n])

In [ ]:
def generate(model, length, n=3): # n - это длина "окна", должна совпадать с аналогичным параметром модели
    current = triplet(model, w='END')
    sentence = [i for i in current]
    for i in range(3, length+1):
        #print(current)
        if tuple(current) in model: 
            current_dictogram = model[tuple(current)]
            random_weighted_word = current_dictogram.return_weighted_random_word()
            #print(random_weighted_word)
            sentence.append(random_weighted_word)
            #print(sentence)
            current = sentence[i-n+1:i+n]
            #print(current)
        else:
            #print(current)
            random_word = triplet(model)
            sentence.extend([i for i in random_word])
            #print(sentence)
            current = sentence[i-n+1:i+n]
            
    sentence = sentence[1:]
    sentence[0] = sentence[0].capitalize()
    for i in range (len(sentence)-1):
        if sentence[i] == 'END':
            sentence[i+1] = sentence[i+1].capitalize()
    res = ' '.join(sentence) + '.'
    res = re.sub(' END', '.',  res)
    return res

t = generate(model, 300, 4)
print(t)

# Библиотека Markovify 

In [1]:
import markovify

# Get raw text as string.
with open('nietzsche.txt') as f:
    text = f.read()

text_model = markovify.Text(text)

In [2]:
for i in range(5):
    print(text_model.make_sentence())

The cruelty to themselves to their interest that this contest should always be kept up in his ears away in the cosmos on a time men sacrificed human beings to their pride, and also unfortunately eager to help and save to an extraordinary pitch of feeling and thoughts, whereas it is comprised in one breath.
In the new warlike age on which the saint suggested to them singing!
Love to one another in respect to all that gave rise to them singing!
I am allowed, a little distance from us.
The resulting form of gratification in which the struggle for the fact that at present on a Cyclops.


In [3]:
for i in range(3):
    print(text_model.make_short_sentence(140))

Thus do many thinkers bring themselves to worship stone, stupidity, gravity, fate, nothingness?
To be sure, people have not disdained to make use of their own personality.
No doubt he has to do that.


In [6]:
text_model = markovify.Text(text, state_size=3)
for i in range(3):
    print(text_model.make_sentence())

None
It seems, therefore, that however little we may imagine ourselves to be as hard punished as Christianity promises he shall be.
One evil only did not fly out of the grave of their primitive antiquity!


# Задание 

Сгенерируйте текст на русском языке по вашей любимой книге:)